In [4]:
# import libaries 
import csv
import numpy as np
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

ModuleNotFoundError: No module named 'tensorflow'

In [7]:
# get csv data
groundtruthdata = []
with open('data.csv', 'r') as csv_datei:
    csv_reader = csv.reader(csv_datei)
    # skip header
    next(csv_reader)
    for zeile in csv_reader:
        groundtruthdata.append(zeile[:2])

In [16]:
# path to images
data_path = 'images/'
categories = ['0', '1'] 

img_size = (240, 240)
batch_size = 32

In [3]:
def load_and_preprocess_image(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.resize(img, img_size)
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

In [28]:
labels = [inner_array[1] for inner_array in groundtruthdata]
file_paths = [data_path + str(inner_array[0]) + '.jpg'  for inner_array in groundtruthdata]

In [29]:
train_paths, test_paths, train_labels, test_labels = train_test_split(file_paths, labels, test_size=0.2, random_state=42)

In [30]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))

train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

In [31]:
train_dataset = train_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [32]:
def dataset_to_numpy(dataset):
    images, labels = [], []
    for img_batch, label_batch in dataset:
        images.extend(img_batch.numpy())
        labels.extend(label_batch.numpy())
    return np.array(images), np.array(labels)

In [33]:
X_train, y_train = dataset_to_numpy(train_dataset)
X_test, y_test = dataset_to_numpy(test_dataset)

In [34]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

In [39]:
#knn = KNeighborsClassifier(n_neighbors=3)

In [3]:
# Define the hyperparameters to be tuned
param_grid_knn = {'n_neighbors': list(range(1, 11)), 'weights': ['uniform', 'distance']}

# Create an instance of KNeighborsClassifier
knn = KNeighborsClassifier()

# Create an instance of GridSearchCV
grid_search_knn = GridSearchCV(knn, param_grid_knn, cv=5)

# Fit the GridSearchCV to the standardized training data
grid_search_knn.fit(X_train, y_train)

# Get the best hyperparameters
best_hyperparameters_knn = grid_search_knn.best_params_

# Print the best hyperparameters
print("Best Hyperparameters for k-NN Classifier:", best_hyperparameters_knn)

#Check the best model with the test set

# Get the best estimator
best_knn_model = grid_search_knn.best_estimator_

# Calculate accuracy rate on the test set
accuracy_test_knn = best_knn_model.score(X_test, y_test) * 100

# Print the result
print("Accuracy rate on the test set for k-NN Classifier:", round(accuracy_test_knn, 2), "%")

NameError: name 'KNeighborsClassifier' is not defined

In [2]:
#knn.fit(X_train, y_train)

In [41]:
#y_pred = knn.predict(X_test)

#accuracy = accuracy_score(y_test, y_pred)
#print(f'Model Accuracy: {accuracy * 100:.2f}%')

Model Accuracy: 94.42%
